In [1]:
#EDA

In [2]:
import torch

In [3]:
#pip install torch==2.0.0
#pip install pysentimiento==0.7.2
#pip install evaluate==0.4.0
#pip install datasets==2.14.5

In [4]:
#!pip install pysentimiento transformers datasets accelerate evaluate

In [5]:
import datasets
import evaluate

2024-03-23 19:55:30.784342: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-23 19:55:30.818834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 19:55:30.818862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 19:55:30.819786: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 19:55:30.825687: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

In [6]:
import numpy as np
from datasets import load_dataset

In [7]:
import ipywidgets as widgets

In [8]:
#Helper Functions

In [9]:
# label to name
def label2name(x):
    if x == 0:
        return "Negative"
    if x == 1:
        return "Neutral"
    if x == 2:
        return "Positive"

In [10]:

f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")

def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)
    
    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="macro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="macro"))
    return results

In [11]:
#Read Data


In [12]:
data_files = {"train": "data/train.csv", "validation": "data/val.csv", "test": "data/test.csv"}
ds = load_dataset("csv", data_files=data_files)

In [13]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_name', 'text_original', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 7594
    })
    validation: Dataset({
        features: ['text', 'label', 'label_name', 'text_original', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 2374
    })
    test: Dataset({
        features: ['text', 'label', 'label_name', 'text_original', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count'],
        num_rows: 1899
    })
})

In [14]:
#push to hugging face

In [15]:
#https://huggingface.co/settings/tokens

In [16]:
# from huggingface_hub import notebook_login
# notebook_login()
# ds.push_to_hub("jairleo95/social-media-peruvian-sentiment")

In [17]:
ds["train"].features

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'label_name': Value(dtype='string', id=None),
 'text_original': Value(dtype='string', id=None),
 'tokenized_text': Value(dtype='string', id=None),
 'sent_token_length': Value(dtype='int64', id=None),
 'sent_bert_token_length': Value(dtype='int64', id=None),
 'char_count': Value(dtype='int64', id=None),
 'Character Count': Value(dtype='int64', id=None)}

In [18]:
ds["test"]["label"][:10]

[1, 0, 0, 0, 0, 2, 1, 0, 2, 2]

In [19]:
df = ds['train'].to_pandas()
df.head()

,text,label,label_name,text_original,tokenized_text,sent_token_length,sent_bert_token_length,char_count,Character Count
0,no yuli con eso se va con pendeivis 😞,1,Neutral,No Yuli con eso se va con pendeivis 😞,no yuli con eso se va con pendeivis 😞,9,12,37,37
1,este es más bruto... cosas que necesita el pe...,0,Negative,Este es más bruto... Cosas que necesita el Pe...,este es más bruto cosas que necesita el pe...,27,35,144,144
2,bakan tu video pero el chambar es riko pero pa...,2,Positive,Bakan tu video pero el chambar es riko pero pa...,bakan tu video pero el chambar es riko pero pa...,53,67,267,267
3,no sabes que hacer para llamar la atención par...,0,Negative,no sabes que hacer para llamar la atención par...,no sabes que hacer para llamar la atención par...,18,20,93,93
4,"pe chino dame chamba mano, almenos pa limpiart...",1,Neutral,"Pe CHINO DAME CHAMBA MANO, ALMENOS PA LIMPIART...",pe chino dame chamba mano almenos pa limpiart...,11,14,61,61


In [20]:
# encode label and mapping label name
#df["label"] = df["label"].apply(lambda x: label_encode(x))
df["label_name"] = df["label"].apply(lambda x: label2name(x))

In [21]:
# clean text, lowercase and remove punk
#df["text"] = df["text"].apply(lambda x: remove_punct(clean(remove_emoji(x).lower())[0][0]))

In [22]:
df.sample(10)

,text,label,label_name,text_original,tokenized_text,sent_token_length,sent_bert_token_length,char_count,Character Count
4049,en canadá si te resulta!! porque uno se la pas...,1,Neutral,En Canadá si te resulta!! Porque uno se la pas...,en canadá si te resulta porque uno se la pas...,55,67,288,288
4605,john coral lázaro tramquilo parroquiano 🤣🤣🤣,0,Negative,John Coral Lázaro tramquilo parroquiano 🤣🤣🤣🤣,john coral lázaro tramquilo parroquiano 🤣🤣🤣,6,12,43,43
5361,no hay tanta arrechura,0,Negative,No hay tanta arrechura,no hay tanta arrechura,4,6,22,22
78,tengo q ir! esas guitarras tristes... 💔,1,Neutral,Tengo q ir! Esas guitarras tristes... 💔,tengo q ir esas guitarras tristes 💔,7,10,39,39
2902,si a esa chica por ese monto ínfimo le quieren...,0,Negative,Si a esa chica por ese monto ínfimo le quieren...,si a esa chica por ese monto ínfimo le quieren...,27,32,141,141
867,número 7️⃣ ya voy como 💯 comentarios ahorita m...,2,Positive,Número 7️⃣ ya voy como 💯 comentarios ahorita m...,número 7️⃣ ya voy como 💯 comentarios ahorita m...,11,12,58,58
5858,jaja la canchita en mi cartera y con gaseosa y...,1,Neutral,Jajjaja la canchita en mi cartera y con gaseos...,jaja la canchita en mi cartera y con gaseosa y...,13,17,64,64
6443,"bravazo el video, muchachos les faltó probar e...",2,Positive,"Bravazo el video, muchachos les faltó probar e...",bravazo el video muchachos les faltó probar e...,23,29,132,132
2155,hanns licera pajuelo leidy huaman astrid rosas...,1,Neutral,Hanns Licera Pajuelo Leidy Huaman Astrid Rosas...,hanns licera pajuelo leidy huaman astrid rosas...,14,27,89,89
3577,el código para el mensaje misio es 4858 y lueg...,1,Neutral,El código para el mensaje misio es 4858 y lueg...,el código para el mensaje misio es 4858 y lueg...,23,26,120,120


In [23]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "pysentimiento/robertuito-sentiment-analysis"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
# columns = ['input_ids', 'attention_mask', 'label']
# ds.set_format(type='torch', columns=columns)
# ds

In [25]:
from pysentimiento.preprocessing import preprocess_tweet
preprocessed_ds = ds.map(lambda ex: {"text": preprocess_tweet(ex["text"], lang="es")})

In [26]:
tokenized_ds = preprocessed_ds.map(
    lambda batch: tokenizer(
        batch["text"], padding=True, truncation=True
        ),
    batched=True, batch_size=32
)

Map:   0%|          | 0/2374 [00:00<?, ? examples/s]

In [27]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_name', 'text_original', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7594
    })
    validation: Dataset({
        features: ['text', 'label', 'label_name', 'text_original', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2374
    })
    test: Dataset({
        features: ['text', 'label', 'label_name', 'text_original', 'tokenized_text', 'sent_token_length', 'sent_bert_token_length', 'char_count', 'Character Count', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1899
    })
})

In [28]:
tokenized_ds['train']['input_ids'][0][::10]

[0, 49, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [29]:
tokenized_ds['train']['attention_mask'][0][::10]

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [30]:
tokenized_ds['train']['text'][10]

'tu hermano gemelo manolo... sobrado la haces como su doble..   emoji cara con mano sobre la boca emoji  emoji cara con mano sobre la boca emoji'

In [31]:
#!pip install ipdb

In [32]:
#import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30002, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [33]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

training_args = TrainingArguments(
    per_device_train_batch_size=32,
    output_dir="trainer_robertuito_sa",
    warmup_ratio=0.1,
    learning_rate=5e-5,
    do_eval=True,
    evaluation_strategy="epoch",
    num_train_epochs=5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

/home/darkstar/anaconda3/envs/robertuito-env/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [34]:
trainer.train()

{'eval_loss': 0.611771285533905, 'eval_f1': 0.725496288732459, 'eval_recall': 0.7243278871953841, 'eval_runtime': 11.3199, 'eval_samples_per_second': 209.718, 'eval_steps_per_second': 26.237, 'epoch': 1.0}
{'eval_loss': 0.6473233103752136, 'eval_f1': 0.7340101227177801, 'eval_recall': 0.7356862710731219, 'eval_runtime': 11.5085, 'eval_samples_per_second': 206.282, 'eval_steps_per_second': 25.807, 'epoch': 2.0}
{'loss': 0.5379, 'grad_norm': 5.051478385925293, 'learning_rate': 3.221288515406163e-05, 'epoch': 2.1}
{'eval_loss': 0.8852047324180603, 'eval_f1': 0.7235017544061685, 'eval_recall': 0.72673855483754, 'eval_runtime': 11.5102, 'eval_samples_per_second': 206.252, 'eval_steps_per_second': 25.803, 'epoch': 3.0}
{'eval_loss': 1.1249061822891235, 'eval_f1': 0.7306746429353659, 'eval_recall': 0.7304955401795366, 'eval_runtime': 10.6459, 'eval_samples_per_second': 222.996, 'eval_steps_per_second': 27.898, 'epoch': 4.0}
{'loss': 0.1432, 'grad_norm': 1.3750090599060059, 'learning_rate': 8.

TrainOutput(global_step=1190, training_loss=0.29305695926441866, metrics={'train_runtime': 590.1811, 'train_samples_per_second': 64.336, 'train_steps_per_second': 2.016, 'train_loss': 0.29305695926441866, 'epoch': 5.0})

In [35]:
#Test on validation set

In [36]:
trainer.evaluate(tokenized_ds["test"])

{'eval_loss': 1.3385035991668701, 'eval_f1': 0.7032715440575292, 'eval_recall': 0.7043530515723176, 'eval_runtime': 8.4523, 'eval_samples_per_second': 224.672, 'eval_steps_per_second': 28.158, 'epoch': 5.0}


{'eval_loss': 1.3385035991668701,
 'eval_f1': 0.7032715440575292,
 'eval_recall': 0.7043530515723176,
 'eval_runtime': 8.4523,
 'eval_samples_per_second': 224.672,
 'eval_steps_per_second': 28.158,
 'epoch': 5.0}

In [37]:
#model.load_state_dict(torch.load(f'./_BERT_epoch_3.model', map_location=torch.device('cpu')))

In [38]:
#Error Analisys

In [39]:
import pandas as pd

In [40]:
val_df = ds['validation'].to_pandas()
#val_df

In [41]:
from tqdm.notebook import tqdm

In [42]:
# step by step predictions on dataframe
# We do this to view predictions in the pandas dataframe and easily filter them and perform error analysis.
pred_final = []

for i, row in tqdm(val_df.iterrows(), total=val_df.shape[0]):
    predictions = []

    text = row["text"]
    encoded_data_test_single = tokenizer.batch_encode_plus([text], 
    # add_special_tokens=config.add_special_tokens, 
    # return_attention_mask=config.return_attention_mask, 
    # pad_to_max_length=config.pad_to_max_length, 
    max_length=128,
    # return_tensors=config.return_tensors
    return_tensors="pt"
    )
    input_ids_test = encoded_data_test_single['input_ids']
    attention_masks_test = encoded_data_test_single['attention_mask']

    
    inputs = {'input_ids':      input_ids_test.to(device),
              'attention_mask':attention_masks_test.to(device),
             }

    with torch.no_grad():        
        outputs = model(**inputs)
    
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    predictions.append(logits)
    predictions = np.concatenate(predictions, axis=0)
    pred_final.append(np.argmax(predictions, axis=1).flatten()[0])

  0%|          | 0/2374 [00:00<?, ?it/s]

In [43]:
# add pred into val_df
val_df["pred"] = pred_final

In [44]:
#  Add control column for easier wrong and right predictions
control = val_df.pred.values == val_df.label.values
val_df["control"] = control

In [45]:
# filtering false predictions
val_df = val_df[val_df.control == False]

In [46]:
# label to intent mapping
name2label = {"Negative":0,
              "Neutral":1,
             "Positive":2
             }
label2name = {v: k for k, v in name2label.items()}

val_df["pred_name"] = val_df.pred.apply(lambda x: label2name.get(x)) 

In [47]:
from sklearn.metrics import confusion_matrix

# We create a confusion matrix to better observe the classes that the model confuses.
pred_name_values = val_df.pred_name.values
label_values = val_df.label_name.values
confmat = confusion_matrix(label_values, pred_name_values, labels=list(name2label.keys()))

In [48]:
confmat

array([[  0, 205,  39],
       [131,   0, 139],
       [ 56, 153,   0]])

In [49]:
df_confusion_val = pd.crosstab(label_values, pred_name_values)
df_confusion_val

col_0,Negative,Neutral,Positive
row_0,,,
Negative,0,205,39
Neutral,131,0,139
Positive,56,153,0


In [50]:
# save confissuan matrix df
df_confusion_val.to_csv("val_df_confusion.csv")

## Inference

In [51]:
tokenized_ds['train']['text'][::30]

['no yuli con eso se va con pendeivis  emoji cara decepcionada emoji',
 'estaba aguja',
 'y del agua su duro  emoji cara sonriendo con ojos sonrientes emoji  emoji cara sonriendo con ojos sonrientes emoji',
 'christian cacho vergara ahí está tu chuleton  emoji cara babeando emoji',
 'malditas lacras deben chaparlos y meterlos bien presos... vienen de otro país a hacer esto, no pues ya basta!!!  emoji cara cabreada emoji  emoji cara cabreada emoji  emoji cara cabreada emoji',
 '@usuario juan antonio castillo solari primo pronto con tu libro  emoji cara radiante con ojos sonrientes emoji  emoji cara feliz con ojos sonrientes emoji',
 'delicioso, muchas gracias por los aportes. vivo en la sierra, exactamente en apurimac y en mi trabajo de empoderar a las mujeres, tambien he inventado o reinventado recetas con quinua, como carapulcra, ensaladas, postres y otros. debemos de valorar nuestros productos  emoji cara feliz con ojos sonrientes emoji  emoji cara feliz con ojos sonrientes emoji',
 

In [52]:
encoded_data_test = tokenizer.batch_encode_plus(
    val_df.text.values, 
    add_special_tokens=config.add_special_tokens, 
    return_attention_mask=config.return_attention_mask, 
    pad_to_max_length=config.pad_to_max_length,
    max_length=128, 
    return_tensors=config.return_tensors
)

NameError: name 'config' is not defined

In [ ]:
#Evaluate the Model Qualitatively (Human Evaluation)

In [ ]:
#Evaluate the Model Quantitatively (with F1 Metric)

In [ ]:
texts = val_df[0:20]['text']
human_baseline_labels = val_df[0:20]['label']

In [ ]:
texts

In [ ]:
human_baseline_labels

In [ ]:
#original_model_summaries = []
model_classifications = []

In [ ]:
for _, text in enumerate(texts):
    input_ids = tokenizer(text, return_tensors="pt", max_length=128).input_ids.to('cuda')
    logits = model(input_ids).logits
    probabilities = logits.softmax(dim=-1).tolist()[0]
    model_classifications.append(np.argmax(probabilities).flatten()[0])

In [ ]:
model_classifications